In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install dicomsdl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 1.5 MB/s eta 0:00:00a 0:00:010m


# Converting images to png

In [4]:
import dicomsdl as dicoml
import cv2
import pydicom

from joblib import Parallel, delayed
import glob
import time
import numpy as np
import os
from matplotlib import pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'dicomsdl'

In [4]:
image_dir_dicomsdl = '/kaggle/working/png_file_dic/'
os.makedirs(image_dir_dicomsdl, exist_ok=True)
IMAGES_TO_PROCESS = 500
train_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/train_images/*/*.dcm")

def process(f, size=512, save_folder=None, extension="png"):    
    patient = f.split('/')[-2]
    image_name = f.split('/')[-1][:-4]
    dicom = dicoml.open(f)
    img = dicom.pixelData()
    img = (img - img.min()) / (img.max() - img.min())
    if dicom.getPixelDataInfo()['PhotometricInterpretation'] == "MONOCHROME1":
        img = 1 - img
    image = (img * 255).astype(np.uint8)
    img = cv2.resize(image, (size, size))
    file_name = f'{save_folder}' + f"{patient}_{image_name}.{extension}"
    cv2.imwrite(file_name, img)

In [5]:
%%time
Parallel(n_jobs=4)(
    delayed(process)(f, size = 512, save_folder = image_dir_dicomsdl)
    for f in train_images[:IMAGES_TO_PROCESS]
)

CPU times: user 4.53 s, sys: 141 ms, total: 4.67 s
Wall time: 3min 56s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Preprocessing

In [25]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [37]:
df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True


In [38]:
df.dtypes

site_id                      int64
patient_id                   int64
image_id                     int64
laterality                  object
view                        object
age                        float64
cancer                       int64
biopsy                       int64
invasive                     int64
BIRADS                     float64
implant                      int64
density                     object
machine_id                   int64
difficult_negative_case       bool
dtype: object

In [39]:
df['file_id'] = df['patient_id'].astype(str) + '_' + df['image_id'].astype(str)
df.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,file_id
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,10006_462822612
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,10006_1459541791
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,10006_1864590858
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,10006_1874946579
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,10011_220375232


In [40]:
file_ids = df['file_id'].values
labels = df['cancer'].values

In [41]:
file_ids

array(['10006_462822612', '10006_1459541791', '10006_1864590858', ...,
       '9989_1078943060', '9989_398038886', '9989_439796429'],
      dtype=object)

In [42]:
labels

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
data = tf.data.Dataset.from_tensor_slices((file_ids, labels))

In [44]:
data.take(1)

<TakeDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [47]:
def load_and_preprocess_image(file_ids, label):
    image_path = f'/kaggle/working/png_file_dic/{file_ids}.png'
    print(image_path)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image)
#     image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
#     image /= 255.0  # normalize to [0,1] range
    return image, label

data = data.map(load_and_preprocess_image)

/kaggle/working/png_file_dic/Tensor("args_0:0", shape=(None, None, None), dtype=uint8).png


In [46]:
data

<MapDataset shapes: ((None, None, None), ()), types: (tf.uint8, tf.int64)>

In [19]:
X, y = data.unbatch()

ValueError: Value Tensor("args_1:0", shape=(), dtype=int64) has insufficient rank for batching.

In [5]:
path = '/kaggle/working/png_file_dic'
data = tf.data.Dataset.list_files(path + '/*.png')

In [6]:
def load_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image)
    return image

data = data.map(load_image)

In [12]:
data = data.batch(batch_size=100)
data = data.shuffle(buffer_size=1000)

In [14]:
def preprocess_image(image):
    image = tf.image.resize(image, (512, 512))
    image /= 255.0  # normalize to [0,1] range
    return image

data = data.map(preprocess_image)

In [15]:
X = data.unbatch()

In [16]:
X

<_UnbatchDataset shapes: (512, 512, None), types: tf.float32>